<a href="https://colab.research.google.com/github/aboelela924/tensorFlowTutorials/blob/master/KerasTunnerTfTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keras-tuner

In [7]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
import IPython

In [9]:
(img_train, labels_train), (img_test, labels_test) =  keras.datasets.fashion_mnist.load_data()

In [4]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

In [16]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))

    learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
    
    return model

In [17]:
tuner = kt.Hyperband(model_builder, "val_loss", 10, 
                     directory="/content/my_dir",
                     project_name="intro_to_kt")

INFO:tensorflow:Reloading Oracle from existing project /content/my_dir/intro_to_kt/oracle.json


In [18]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*arg, **kwargs):
        IPython.display.clear_output(wait=True)

In [20]:
tuner.search(img_train, labels_train, epochs=10, 
             validation_data=(img_test, labels_test), 
             callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=10)


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")